In [1]:
from pathlib import Path
from topagnps2cche1d import Watershed, CrossSection

# root_dir = Path("../data_inputs/goodwin_creek_1m/")
# root_dir = Path("../data_inputs/goodwin_creek_1m_1_and_3_reaches/TopAGNPS_DataSets/3_Reach")
root_dir = Path("../data_inputs/topagnps_ohio_files/")

path_to_reaches = root_dir / "AnnAGNPS_Reach_Data_Section.csv"
path_to_cells = root_dir / "AnnAGNPS_Cell_Data_Section.csv"
path_to_reaches_raster = root_dir / "AnnAGNPS_Reach_IDs.asc"

In [2]:
watershed = Watershed()
watershed.import_topagnps_reaches_network(path_to_reaches)
watershed.import_topagnps_cells(path_to_cells)

watershed.read_reaches_geometry_from_topagnps_asc_file(path_to_reaches_raster)

watershed.assign_strahler_number_to_reaches()
watershed.ignore_reaches_with_strahler_leq(0)
# watershed.reaches[4].ignore_reach()

# watershed.update_graph()
# watershed.update_junctions_and_node_types()
# watershed.identify_inflow_sources()

# watershed.renumber_all_nodes_and_reaches_in_CCHE1D_computational_order()
# watershed.set_node_id_to_compute_id()

watershed.update_watershed()

print("Resampling reaches")
watershed.resample_reaches(id_list="all", numpoints=5)

watershed.assign_cross_section_to_all_points()

Reading Reaches: 100%|██████████| 27/27 [00:00<00:00, 126.40it/s]

Resampling reaches


In [7]:
watershed.plot()

:Overlay
   .NdOverlay.I  :NdOverlay   [Reach_ID]
      :Curve   [X]   (Y,TYPE,US2ID,USID,ID,COMPUTEID,DSID,CCHE1D_ID,Reach_ID)
   .NdOverlay.II :NdOverlay   [Reach_ID]
      :Scatter   [X]   (Y,TYPE,US2ID,USID,ID,COMPUTEID,DSID,CCHE1D_ID,Reach_ID)

In [19]:
for reach in watershed.reaches.values():
    print(reach)

-----------------------------
Reach            : 2
CCHE1D Ch. ID    : 13
Receiving Reach  : 1
Slope            : 1e-05
Strahler Number  : 3
Ignore?          : False
(US/DS) Nodes    : (11, 12)
-----------------------------
Reach            : 3
CCHE1D Ch. ID    : 12
Receiving Reach  : 2
Slope            : 0.00097
Strahler Number  : 3
Ignore?          : False
(US/DS) Nodes    : (133, 137)
-----------------------------
Reach            : 4
CCHE1D Ch. ID    : 11
Receiving Reach  : 3
Slope            : 0.00251
Strahler Number  : 2
Ignore?          : False
(US/DS) Nodes    : (None, None)
-----------------------------
Reach            : 5
CCHE1D Ch. ID    : None
Receiving Reach  : 4
Slope            : 1e-05
Strahler Number  : 1
Ignore?          : True
(US/DS) Nodes    : (None, None)
-----------------------------
Reach            : 6
CCHE1D Ch. ID    : None
Receiving Reach  : 4
Slope            : 1e-05
Strahler Number  : 1
Ignore?          : True
(US/DS) Nodes    : (None, None)
---------------

In [8]:
df_nodes = watershed.create_cche1d_nodes_df()
df_ch = watershed.create_cche1d_channels_df()
df_lk, df_rc = watershed.create_cche1d_links_and_reaches_df()
df_csec, df_csprf = watershed.create_cche1d_csec_csprf_df()
df_tw = watershed.create_cche1d_twcells_df()
df_inflow_reaches = watershed.get_list_of_inflow_reaches_df()

In [9]:
df_nodes

,ND_ID,ND_FRMNO,ND_TYPE,ND_XC,ND_YC,ND_DSID,ND_USID,ND_US2ID,ND_CSID,ND_RSID,ND_STID
0,1,1,0,716666.897,4475720.33,2,-1,-1,1386,-1,1
1,2,2,6,716666.897,4475717.33,3,1,-1,1387,-1,1
2,3,3,6,716663.897,4475714.33,4,2,-1,1388,-1,1
3,4,4,6,716663.897,4475711.33,5,3,-1,1389,-1,1
4,5,5,6,716660.897,4475708.33,6,4,-1,1390,-1,1
...,...,...,...,...,...,...,...,...,...,...,...
1434,1435,1435,6,717170.897,4475999.33,1436,1434,-1,4,-1,1
1435,1436,1436,6,717170.897,4476002.33,1437,1435,-1,3,-1,1
1436,1437,1437,3,717167.897,4476005.33,1438,1436,-1,78,-1,1
1437,1438,1438,2,717167.897,4476005.33,1439,1437,575,2,-1,1


In [10]:
# step = 5
# strahler = 2
# casename = f"goodwin_creek_1m_strahler_gt{strahler}_step_{step}m"
step = "native_"
strahler = 0
casename = f"ohio_strahler_gt{strahler}_step_{step}m"
watershed.write_cche1d_dat_files(
    casename=casename, output_folder=f"./../data_outputs/{casename}"
)

## Plotting network

In [9]:
watershed.plot(
    frame_height=700, frame_width=900, line_width=2, by="CCHE1D_ID", title="Watershed"
)

:Overlay
   .NdOverlay.I  :NdOverlay   [CCHE1D_ID]
      :Curve   [X]   (Y,TYPE,US2ID,USID,ID,COMPUTEID,DSID,CCHE1D_ID,Reach_ID)
   .NdOverlay.II :NdOverlay   [CCHE1D_ID]
      :Scatter   [X]   (Y,TYPE,US2ID,USID,ID,COMPUTEID,DSID,CCHE1D_ID,Reach_ID)

### Cross section

In [10]:
import numpy as np

ws = np.linspace(-10, 10, 15)
zs = 0.02 * ws**2

crosssection = CrossSection(id=1, ws=ws, zs=zs)

In [11]:
ws

array([-10.        ,  -8.57142857,  -7.14285714,  -5.71428571,
        -4.28571429,  -2.85714286,  -1.42857143,   0.        ,
         1.42857143,   2.85714286,   4.28571429,   5.71428571,
         7.14285714,   8.57142857,  10.        ])

In [12]:
crosssection.rbt_idx

15

In [13]:
crosssection.plot(marker_size=7)

:Overlay
   .Curve.I   :Curve   [x]   (y)
   .Scatter.I :Scatter   [x]   (y)